# Convolutional Neural Networks

In [2]:
import torch
import torchvision
from torchvision import transforms
image_path = 'data/'
transform = transforms.Compose([
    transforms.ToTensor(),
])
mnist_train = torchvision.datasets.MNIST(root=image_path, train=True, download=True, transform=transform)
mnist_test = torchvision.datasets.MNIST(root=image_path, train=False, download=True, transform=transform)

train_dl = torch.utils.data.DataLoader(mnist_train, batch_size=100, shuffle=True)
test_dl = torch.utils.data.DataLoader(mnist_test, batch_size=100, shuffle=False)

In [5]:
model = torch.nn.Sequential()

model.add_module(
    'conv2d_1',
    torch.nn.Conv2d(
        in_channels=1, # 1 canal de entrada
        out_channels=32, # 32 canales de salida
        kernel_size=5, # kernel 5x5
        padding=2 # padding 2
        )
    )
model.add_module('relu_1', torch.nn.ReLU()) 
model.add_module('maxpool_1', torch.nn.MaxPool2d(kernel_size=2, stride=2))

model.add_module('conv2d_2', torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding=2))
model.add_module('relu_2', torch.nn.ReLU())
model.add_module('maxpool_2', torch.nn.MaxPool2d(kernel_size=2, stride=2))

model.add_module('flatten', torch.nn.Flatten())
model.add_module('linear_1', torch.nn.Linear(64*7*7, 1024)) # 64 out channels, 
model.add_module('relu_3', torch.nn.ReLU())
model.add_module('dropout', torch.nn.Dropout(p=0.5)) # evitar overfitting
model.add_module('linear_2', torch.nn.Linear(1024, 10))

model

Sequential(
  (conv2d_1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu_1): ReLU()
  (maxpool_1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu_2): ReLU()
  (maxpool_2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_1): Linear(in_features=3136, out_features=1024, bias=True)
  (relu_3): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (linear_2): Linear(in_features=1024, out_features=10, bias=True)
)

In [ ]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    for i, (images, labels) in enumerate(train_dl):
        y = model(images)
        l = loss(y, labels)
        l.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f'epoch: {epoch}, batch: {i}, loss: {l.item()}')

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_dl:
        y = model(images)
        _, predicted = torch.max(y, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'accuracy: {correct / total}')